# How to fit a model

To fit a model, there are 3 main things to specify:

**1. Data:**
The `data`, which should be in the form of a pandas DataFrame.

Different model classes might require different columns in the data. You should check in the [API Reference](https://rlssm.readthedocs.io/en/latest/models/model_classes.html) of each model class (or using `model.fit?`) what the required data columns are.

**2. The priors (optional):**
You can decide whether to use the default priors (which you can see after initializing the model) or whether you want to change the mean or SD of the prior or hyper-prior distributions. Whether you changed the priors or not, they are always printed out when the model starts fitting.

**3. Sampling parameters:**
The sampling parameters **(number of chains, iterations, warmups, thinning, etc.)** are the arguments to the `pystan.StanModel.sampling()` function, and we simply refer to the [pystan documentation](https://pystan.readthedocs.io/) for a better overview.

**Additional learning parameters:**
While all sequential sampling models (DDM and race models) **without a learning component** only require a `data` argument, all models with a learning components (RL models, RLDDMs, and RL+race models) also require a `K` argument, which is the total number of different options in a learning block (note that this can be different from the number of options presented in each trial), and `initial_value_learning`, which is the initial Q value (before learning).

In [10]:
from rlssm.model.models_DDM import DDModel
from rlssm.model.models_RL import RLModel_2A
from rlssm.utility.load_data import load_example_dataset
from rlssm.utility.utils import load_model_results

## Non-learning example (non-hierarchical, simulated data)

In [11]:
model_ddm = DDModel(hierarchical_levels=1)

Using cached StanModel


In [12]:
# simulate some DDM data:
from rlssm.random.random_DDM import simulate_ddm
data_ddm = simulate_ddm(
    n_trials=400, 
    gen_drift=.8, 
    gen_threshold=1.3, 
    gen_ndt=.23)

For the simple, non-hierarchical DDM, it is only necessary to have `rt` and `accuracy` columns:

- *rt*, response times in seconds.

- *accuracy*, 0 if the incorrect option was chosen, 1 if the correct option was chosen.

In [13]:
data_ddm.head()

drift  rel_sp  threshold   ndt     rt  accuracy
participant trial                                                 
1           1        0.8     0.5        1.3  0.23  1.176       1.0
            2        0.8     0.5        1.3  0.23  0.842       1.0
            3        0.8     0.5        1.3  0.23  0.946       0.0
            4        0.8     0.5        1.3  0.23  0.528       0.0
            5        0.8     0.5        1.3  0.23  0.730       1.0

In [14]:
# Run 2 chains, with 2000 samples each, 1000 of which warmup, with custom priors:
model_fit_ddm = model_ddm.fit(
    data_ddm,
    drift_priors={'mu':.5, 'sd':1},
    threshold_priors={'mu':0, 'sd':.5},
    ndt_priors={'mu':0, 'sd':.1},
    chains=2,
    iter_warmup=1000,
    iter_sampling=2000,
    thin=1)

14:12:07 - cmdstanpy - INFO - CmdStan start processing


Fitting the model using the priors:
drift_priors {'mu': 0.5, 'sd': 1}
threshold_priors {'mu': 0, 'sd': 0.5}
ndt_priors {'mu': 0, 'sd': 0.1}


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

14:12:24 - cmdstanpy - INFO - CmdStan done processing.
14:12:24 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: wiener_lpdf: Random variable  = 1.176, but must be greater than nondecision time = 1.32152 (in '/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/DDM/DDM.stan', line 43, column 1 to column 59)
	Exception: wiener_lpdf: Random variable  = 1.176, but must be greater than nondecision time = 1.85439 (in '/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/DDM/DDM.stan', line 43, column 1 to column 59)
	Exception: wiener_lpdf: Random variable  = 1.176, but must be greater than nondecision time = 1.79002 (in '/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/DDM/DDM.stan', line 43, column 1 to column 59)
	Exception: wiener_lpdf: Random variable  = 0.842, but must be greater than nondecision time = 0.962284 (in '/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/DDM/DDM.stan', line 43, column 1 to column 59)
	Exception: wiener_lpdf: Random variable  = 1.1


Checks MCMC diagnostics:
n_eff / iter looks reasonable for all parameters
0 of 4000 iterations saturated the maximum tree depth of 10 (0.0%)
E-BFMI indicated no pathological behavior
0.0 of 4000 iterations ended with a divergence (0.0%)


## Learning example (hierarchical, real data)

In [15]:
model_rl = RLModel_2A(hierarchical_levels = 2)

Using cached StanModel


In [16]:
# import some example data:
data_rl = load_example_dataset(hierarchical_levels = 2)

data_rl.head()

,participant,block_label,trial_block,f_cor,f_inc,cor_option,inc_option,times_seen,rt,accuracy
0,1,1,1,43,39,2,1,1,1.244082,0
1,1,1,2,60,50,4,3,1,1.101821,1
2,1,1,3,44,36,4,2,2,1.029923,0
3,1,1,4,55,55,4,3,2,1.368007,0
4,1,1,5,52,49,4,3,3,1.039329,1


Since this learning model is only fit on choices, `rt` are not required.

Other columns/indexes that should be included are:

- *accuracy*, 0 if the incorrect option was chosen, 1 if the correct option was chosen.

- *trial_block*, the number of trial in a learning session. Should be integers starting from 1.

- *f_cor*, the output from the correct option in the presented pair (the option with higher outcome on average).

- *f_inc*, the output from the incorrect option in the presented pair (the option with lower outcome on average).

- *cor_option*, the number identifying the correct option in the presented pair (the option with higher outcome on average).

- *inc_option*, the number identifying the incorrect option in the presented pair(the option with lower outcome on average).

- *block_label*, the number identifying the learning session. Should be integers starting from 1. Set to 1 in case there is only one learning session.

If the model is hierarchical, also include:

- *participant*, the participant number. Should be integers starting from 1.

If increasing_sensitivity is True, also include:

- *times_seen*, average number of times the presented options have been seen in a learning session.

In [17]:
data_rl.head()

,participant,block_label,trial_block,f_cor,f_inc,cor_option,inc_option,times_seen,rt,accuracy
0,1,1,1,43,39,2,1,1,1.244082,0
1,1,1,2,60,50,4,3,1,1.101821,1
2,1,1,3,44,36,4,2,2,1.029923,0
3,1,1,4,55,55,4,3,2,1.368007,0
4,1,1,5,52,49,4,3,3,1.039329,1


In [18]:
# Run 2 chains, with 3000 samples each, 1000 of which warmup, with thinning and custom priors:
model_fit_rl = model_rl.fit(
    data_rl,
    K=4,
    initial_value_learning=27.5,
    alpha_priors={'mu_mu':-.3, 'sd_mu':.1, 'mu_sd':0, 'sd_sd':.1},
    sensitivity_priors={'mu_mu':-.1, 'sd_mu':.1, 'mu_sd':0, 'sd_sd':.1},
    chains=2,
    iter_sampling=3000,
    iter_warmup=1000,
    print_diagnostics=False, # (not suggested, see below)
    thin=2)

14:12:48 - cmdstanpy - INFO - CmdStan start processing


Fitting the model using the priors:
alpha_priors {'mu_mu': -0.3, 'sd_mu': 0.1, 'mu_sd': 0, 'sd_sd': 0.1}
sensitivity_priors {'mu_mu': -0.1, 'sd_mu': 0.1, 'mu_sd': 0, 'sd_sd': 0.1}


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

14:18:41 - cmdstanpy - INFO - CmdStan done processing.
14:18:41 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: bernoulli_lpmf: Probability parameter[1] is -nan, but must be in the interval [0, 1] (in '/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/RL_2A/hierRL_2A.stan', line 80, column 1 to column 36)
	Exception: bernoulli_lpmf: Probability parameter[1] is -nan, but must be in the interval [0, 1] (in '/home/andrei/PycharmProjects/rlssm/rlssm/stan_models/RL_2A/hierRL_2A.stan', line 80, column 1 to column 36)
Consider re-running with show_console=True if the above output is unclear!


## Diagnostics

As you can see, the MCMC diagnostics are already printed by default (if you do not want this, you can set `print_diagnostics` to `False`). I refer to https://mc-stan.org/users/documentation/case-studies/divergences_and_bias.html for an excellent explanation of what these diagnostics actually mean and how to assess them.

On top of these, you can also check the convergence of the chains and the WAIC:

In [19]:
model_fit_ddm.rhat

,rhat,variable
name,,
lp__,0.999848,lp__
drift,0.999679,drift
threshold,1.000200,threshold
ndt,0.999675,ndt
drift_ll[1],0.999679,drift_ll[1]
...,...,...
log_lik[396],1.000230,log_lik[396]
log_lik[397],0.999938,log_lik[397]
log_lik[398],1.000220,log_lik[398]


In [20]:
model_fit_rl.rhat.describe()

,rhat
count,12814.000000
mean,1.000014
std,0.000573
min,0.999336
25%,0.999582
50%,0.999870
75%,1.000270
max,1.003950


In [21]:
model_fit_ddm.waic

{'lppd': -210.84588753511633,
 'p_waic': 3.3902554935310523,
 'waic': 428.47228605729475,
 'waic_se': 47.0241656587446}

In [22]:
model_fit_rl.waic

{'lppd': -2632.9513369711012,
 'p_waic': 52.72743982796608,
 'waic': 5371.3575535981345,
 'waic_se': 94.06933293481356}

If you want to also see the point-wise WAIC, you can set `pointwise_waic` to `True`.

## Save the results

By default, the model fit results are saved in the same folder, using the `model_label` as filename. you can specify a different location using the `filename` argument.

In [23]:
model_fit_ddm.to_pickle()

Saving file as: /home/andrei/PycharmProjects/rlssm/docs/notebooks/DDM.pkl


In [24]:
model_fit_rl.to_pickle()

Saving file as: /home/andrei/PycharmProjects/rlssm/docs/notebooks/hierRL_2A.pkl


## Re-load previously saved results

In [25]:
model_fit_ddm = load_model_results('DDM.pkl')

In [26]:
model_fit_rl = load_model_results('hierRL_2A.pkl')

The data the model was fit on are stored in `data_info`:

In [27]:
model_fit_rl.data_info['data']

,index,participant,block_label,trial_block,f_cor,f_inc,cor_option,inc_option,times_seen,rt,accuracy
0,0,1,1,1,43,39,2,1,1,1.244082,0
1,1,1,1,2,60,50,4,3,1,1.101821,1
2,2,1,1,3,44,36,4,2,2,1.029923,0
3,3,1,1,4,55,55,4,3,2,1.368007,0
4,4,1,1,5,52,49,4,3,3,1.039329,1
...,...,...,...,...,...,...,...,...,...,...,...
6459,6459,27,3,76,37,36,2,1,39,1.875327,1
6460,6460,27,3,77,58,41,4,2,39,1.696957,1
6461,6461,27,3,78,64,49,4,3,38,2.059956,1
6462,6462,27,3,79,44,37,3,1,39,1.623731,1


The priors are stored in `priors`:

In [46]:
model_fit_ddm.priors

AttributeError: 'DDModelResults' object has no attribute 'priors'

In [30]:
model_fit_rl.priors

AttributeError: 'RLModelResults_2A' object has no attribute 'priors'

And different parameter information are stored in `parameter_info`:

In [31]:
model_fit_ddm.parameters_info

{'hierarchical_levels': 1,
 'n_parameters_individual': 3,
 'n_parameters_trial': 0,
 'n_posterior_samples': 2000,
 'parameters_names': ['drift', 'threshold', 'ndt'],
 'parameters_names_transf': ['transf_drift', 'transf_threshold', 'transf_ndt'],
 'parameters_names_all': ['drift', 'threshold', 'ndt']}

In [32]:
model_fit_rl.waic

{'lppd': -2632.9513369711012,
 'p_waic': 52.72743982796608,
 'waic': 5371.3575535981345,
 'waic_se': 94.06933293481356}

And, of course, you can inspect the model's posteriors, see [How to inspect a model](https://rlssm.readthedocs.io/en/latest/notebooks/inspect_model.html) for more details on this.